In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),'../..')))
import scipy
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn

from Train_fun import train_fun
from Rmse_loss import rmse_loss
from Random_choice import random_choice
from All import all0
from Symmetry import sparse_reg1
from Symmetry import extension
from Select import select

torch.set_default_dtype(torch.float32)
device = torch.device('cuda:0')
np.set_printoptions(suppress=True)
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
class FiveNet_sin(nn.Module):
    def __init__(self, ni):
        super().__init__()
        self.linear1 = nn.Linear(ni, 128)
        self.linear2 = nn.Linear(128,128)
        self.linear3 = nn.Linear(128, 64)
        self.linear4 = nn.Linear(64,64)
        self.linear5 = nn.Linear(64,1)

    def forward(self, x):
        x = torch.sin(self.linear1(x))
        x = torch.sin(self.linear2(x))
        x = torch.sin(self.linear3(x))
        x = torch.sin(self.linear4(x))
        x = self.linear5(x)
        return x

In [3]:
def diff_RD(net,cor_train):
    cache = np.zeros((1,7))
    n_iter = len(cor_train)//10000
    for i in range(n_iter+1):
        pts = cor_train[i*10000:(i+1)*10000].clone().detach().to(device)
        pts.requires_grad_(True)
        outs = net(pts)
        grad = torch.autograd.grad(outs, pts, grad_outputs=torch.ones_like(outs), create_graph=True)[0]
        ut_t = grad[:,0].reshape(-1,1) 
        ux_t = grad[:,1].reshape(-1,1)
        uy_t = grad[:,2].reshape(-1,1)
        uxx_t= (torch.autograd.grad(grad[:,1].reshape(-1,1), pts, 
                grad_outputs=torch.ones_like(outs), create_graph=True)[0])[:,1].reshape(-1,1)
        uxy_t= (torch.autograd.grad(grad[:,1].reshape(-1,1), pts, 
                grad_outputs=torch.ones_like(outs), create_graph=True)[0])[:,2].reshape(-1,1)
        uyy_t= (torch.autograd.grad(grad[:,2].reshape(-1,1), pts, 
                grad_outputs=torch.ones_like(outs), create_graph=True)[0])[:,2].reshape(-1,1)
        uyy = uyy_t.cpu().detach().numpy()    
        uxx = uxx_t.cpu().detach().numpy()
        uxy = uxy_t.cpu().detach().numpy()
        ux = ux_t.cpu().detach().numpy()
        uy = uy_t.cpu().detach().numpy()
        ut = ut_t.cpu().detach().numpy()
        u_pred = outs.cpu().detach().numpy()
        del ux_t, ut_t, uy_t, grad, uxx_t, uyy_t, uxy_t, outs
        cache = np.vstack((cache,np.hstack((u_pred,ux,uy,uxx,uxy,uyy,ut))))
    return cache[1:]

In [4]:
data = scipy.io.loadmat('reaction_diffusion_standard.mat') # grid 256*256*201

t_steps = np.real(data['t'].flatten()[:,None])
x_steps = np.real(data['x'].flatten()[:,None])
y_steps = np.real(data['y'].flatten()[:,None])
Exact_u = data['u']
Exact_v = data['v']

X, Y, T = np.meshgrid(x_steps, y_steps, t_steps)

X_star = np.hstack((T.flatten()[:,None],X.flatten()[:,None], Y.flatten()[:,None]))
u_star = Exact_u.flatten()[:,None] 
v_star = Exact_v.flatten()[:,None]   

In [5]:
xx, yy, tt = np.meshgrid(x_steps[1:-1], y_steps[1:-1], t_steps[1:-1])
X_star = np.hstack((tt.flatten()[:,None], xx.flatten()[:,None], yy.flatten()[:,None]))
U_star = np.hstack((Exact_u[1:-1,1:-1,1:-1].flatten()[:,None], 
                        Exact_v[1:-1,1:-1,1:-1].flatten()[:,None]))
data = np.hstack((X_star,U_star))

In [7]:
data = data[data[:,0]<5]
data = data[np.abs(data[:,1])<4]
data = data[np.abs(data[:,2])<4]
#data = data[data[:,0]/t_l%2==0]
#data = data[data[:,1]/x_l%2==0]
#data = data[data[:,2]/y_l%2==0]
data = data.astype('float32')
u = data[:,3].reshape(-1,1)
v = data[:,4].reshape(-1,1)

In [10]:
per = 0.2
np.random.seed(0)
cor_train = torch.from_numpy(data[:,:3])

u_clean = data[:,3].reshape(-1,1)
u_noise = u_clean + per*np.std(u_clean)*np.random.randn(u_clean.shape[0],u_clean.shape[1])
u_noise = u_noise.astype('float32')
u_train = torch.from_numpy(u_noise).reshape(-1,1)

v_clean = data[:,4].reshape(-1,1)
v_noise = v_clean + per*np.std(v_clean)*np.random.randn(v_clean.shape[0],v_clean.shape[1])
v_noise = v_noise.astype('float32')
v_train = torch.from_numpy(v_noise).reshape(-1,1)

In [16]:
netu = FiveNet_sin(3).to(device)
try:
    netu.load_state_dict(torch.load('net/netu_{}'.format(per)))
except:
    train_fun(netu,cor_train,u_train,N_red_lr=4,epochs=5000,lr=0.001,threshold=-1)
    torch.save(netu.state_dict(),'net/netu_{}'.format(per))
    
netv = FiveNet_sin(3).to(device)
try:
    netv.load_state_dict(torch.load('net/netv_{}'.format(per)))
except:
    train_fun(netv,cor_train,v_train,N_red_lr=4,epochs=5000,lr=0.001,threshold=-1)
    torch.save(netv.state_dict(),'net/netv_{}'.format(per))

In [17]:
cache = diff_RD(netu,cor_train)
u_pred = cache[:,0].reshape(-1,1)
ux = cache[:,1].reshape(-1,1)
uy = cache[:,2].reshape(-1,1)
uxx = cache[:,3].reshape(-1,1)
uxy = cache[:,4].reshape(-1,1)
uyy = cache[:,5].reshape(-1,1)
ut = cache[:,-1].reshape(-1,1)

cache = diff_RD(netv,cor_train)
v_pred = cache[:,0].reshape(-1,1)
vx = cache[:,1].reshape(-1,1)
vy = cache[:,2].reshape(-1,1)
vxx = cache[:,3].reshape(-1,1)
vxy = cache[:,4].reshape(-1,1)
vyy = cache[:,5].reshape(-1,1)
vt = cache[:,-1].reshape(-1,1)

In [18]:
rmse_loss(0.1*(uxx+uyy)-u_pred*v_pred**2-u_pred**3+v_pred**3+u_pred**2*v_pred+u_pred,ut)

tensor(0.0319, device='cuda:0', dtype=torch.float64)

In [24]:
# make variable institution accroding the u_result_pre
cache = np.hstack((v_pred,u_pred,ux,uy,uxx,uxy,uyy,ut))
cache = random_choice(cache,50000)
# choose new variable
x = extension(cache[:,:-1],deg=3,state=2)
lin_reg = sparse_reg1(cache,deg=3,state=2,tol=0.05,alpha=10**-5)
x_feature = x.T[np.where(np.abs(lin_reg.coef_)>0,True,False)].T
feature = np.hstack((x_feature,cache[:,-1].reshape(-1,1)))

0 tensor(0.0333, device='cuda:0', dtype=torch.float64)
1 tensor(0.0297, device='cuda:0', dtype=torch.float64)
2 tensor(0.0366, device='cuda:0', dtype=torch.float64)
3 tensor(0.0333, device='cuda:0', dtype=torch.float64)
4 tensor(0.0333, device='cuda:0', dtype=torch.float64)
5 tensor(0.0333, device='cuda:0', dtype=torch.float64)
6 tensor(0.0358, device='cuda:0', dtype=torch.float64)


In [33]:
state  = feature.shape[1] - 1
print(state)

7


In [29]:
LC, exp_list, exp = all0('resultu_{}.txt'.format(per),feature,state=state,deg=2,
                         compare_threshold=1,tol=0.05,alpha=10**-5,
                         niterations=300, data_points=500)

Compiling Julia backend...
Started!

Expressions evaluated per second: 7.030e+05
Head worker occupation: 9.2%
Progress: 1599 / 4500 total iterations (35.533%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           5.053e-02  1.594e+01  y = x₁
3           2.154e-03  1.578e+00  y = (x₁ + x₃)
5           1.546e-03  1.658e-01  y = ((x₃ + x₁) * 1.043)
7           5.506e-04  5.162e-01  y = ((x₂ * (x₀ / x₃)) * 0.9239)
9           5.258e-04  2.312e-02  y = (x₀ * ((x₂ / (x₃ * 1.0823)) + -0.0074576))
11          5.173e-04  8.066e-03  y = (x₀ * (((0.9239 / x₃) + (-0.12509 * x₄)) * x₂))
13          5.169e-04  4.539e-04  y = (x₂ * (((0.92397 / x₃) - ((x₂ * x₄) * x₄)) * x₀))
14          5.137e-04  6.241e-03  y = (x₂ * (((0.92397 / x₃) - cube(x₄ * 0.83245)) * x₀))
16          5.115e-04  2.107e-03  y = (x₂ * (((0.92397 / x₃) - (cube(x₄ * x₀) * x₄)) * x₀))
18          5.025e-04  8.905e-03  y

0 tensor(0.0311, device='cuda:0', dtype=torch.float64)
1 tensor(0.0755, device='cuda:0', dtype=torch.float64)
2 tensor(0.6191, device='cuda:0', dtype=torch.float64)
3 tensor(0.2955, device='cuda:0', dtype=torch.float64)
4 tensor(0.0755, device='cuda:0', dtype=torch.float64)
5 tensor(0.0651, device='cuda:0', dtype=torch.float64)
[ 0.          0.87585313  1.00380439  0.99265933 -0.86770188  0.09238965
  0.09428967  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.        ]
[[0.         1.         0.07159583 1.05177272 0.66888494 0.67120495]
 [0.         2.         0.05057298 0.80900022 0.38015529 0.38947374]
 [0.         3.         1.08340809 0.00506502 0.88582922 0.05552232]
 [0.         4.         0.50249706 0.62101064 0.51025305 0.44220513]
 [0.         5.         0.50154332 0.62255808 0.51330392 0.43919502]
 [1.   

1 tensor(0.0755, device='cuda:0', dtype=torch.float64)
2 tensor(0.8094, device='cuda:0', dtype=torch.float64)
3 tensor(0.0755, device='cuda:0', dtype=torch.float64)
[ 0.          0.8751842   1.0010182  -0.86702873  0.09329985  0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.        ]
[[0.         1.         0.07809324 1.16438108 0.66318074 0.67306453]
 [0.         2.         1.08257415 0.00506764 0.88514783 0.05548416]
 [0.         3.         0.66364468 0.82202603 0.5482076  0.40841305]
 [1.         2.         1.08687497 0.07795832 0.62673872 0.61592301]
 [1.         3.         0.73454516 0.88554564 0.45160789 0.50344973]
 [2.         3.         0.77786132 0.6267165  0.34256113 0.48299615]]
[[0.         1.         0.07809324 0.11775559 0.        ]
 [0.         2.         0.00506764 0.09133491 1.        ]
 [0.         3.         0.40841305 0.74499707 3.        ]
 [1.         2.         0.07795832 0.12657154 1.        ]
 [1.         3.     

Started!

Expressions evaluated per second: 7.660e+05
Head worker occupation: 8.2%
Progress: 1686 / 4500 total iterations (37.467%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           3.629e-01  1.594e+01  y = 0.02493
3           3.613e-01  2.200e-03  y = (0.0028891 / x₀)
5           3.606e-01  1.034e-03  y = ((0.0033057 / x₀) - -0.030475)
7           3.602e-01  5.150e-04  y = ((x₀ + (0.034927 / x₀)) * 0.079011)
8           3.589e-01  3.648e-03  y = (sin(x₀ / -0.19619) * -0.093355)
10          3.580e-01  1.177e-03  y = ((sin(x₀ / 0.20196) + 0.30713) * 0.08596)
12          3.539e-01  5.822e-03  y = (sin(x₀ / (-0.72365 + x₀)) * (0.22098 / -1.0684))
15          3.535e-01  4.052e-04  y = (sin(sin(x₀) / (-0.72365 + x₀)) * (0.22098 / -1.3721))
17          3.529e-01  8.264e-04  y = (sin((sin(x₀) / (-0.72365 + x₀)) * 0.9612) * (0.22098 / -1...
                                  .3

In [32]:
k = select(LC,0.05)
exp_list[k]

[-1  1  1  1  1  1  1  1  1  1]
[1 1 1 1 1 1 1 1 1]
[-1 -1 -1 -1 -1 -1 -1 -1]


x0 + 0.09819781*x1